In [3]:
# Predictive Infrastructure Demand Forecasting
# SDG 9 - Infrastructure Development

# Install required packages (run this first in Colab)
# !pip install prophet geopandas folium plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For time series forecasting
from prophet import Prophet

# For spatial analysis
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

# For interactive plots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

print(" All libraries imported successfully!")
print(" Predictive Infrastructure Demand Forecasting System")
print(" SDG 9: Industry, Innovation and Infrastructure")



def generate_grid_data():
    """Generate synthetic grid-level infrastructure data"""
    print("\n Generating synthetic grid-level data...")

    # Create 25 grid cells (5x5) representing different urban areas
    grid_ids = []
    coordinates = []
    population = []
    area_types = []

    np.random.seed(42)

    for i in range(5):
        for j in range(5):
            grid_id = f"GRID_{i}_{j}"
            grid_ids.append(grid_id)

            # Coordinates (simulating a city grid)
            lat = 40.7589 + i * 0.01  # Around NYC coordinates
            lon = -73.9851 + j * 0.01
            coordinates.append((lat, lon))

            # Population varies by location (higher in center)
            center_distance = np.sqrt((i-2)**2 + (j-2)**2)
            base_pop = max(1000, 5000 - center_distance * 800)
            pop = int(base_pop + np.random.normal(0, 200))
            population.append(pop)

            # Area types
            if center_distance < 1.5:
                area_types.append('Urban_Core')
            elif center_distance < 3:
                area_types.append('Suburban')
            else:
                area_types.append('Rural')

    grid_df = pd.DataFrame({
        'grid_id': grid_ids,
        'latitude': [coord[0] for coord in coordinates],
        'longitude': [coord[1] for coord in coordinates],
        'population_2024': population,
        'area_type': area_types
    })

    return grid_df

def generate_historical_demand():
    """Generate historical infrastructure demand data"""
    print(" Generating historical demand data (2019-2024)...")

    # Date range
    start_date = datetime(2019, 1, 1)
    end_date = datetime(2024, 12, 31)
    dates = pd.date_range(start_date, end_date, freq='M')

    # Get grid data
    grid_df = generate_grid_data()

    all_data = []

    for _, grid in grid_df.iterrows():
        grid_id = grid['grid_id']
        base_pop = grid['population_2024']
        area_type = grid['area_type']

        # Base demand multipliers by area type
        multipliers = {
            'Urban_Core': {'water': 1.5, 'electricity': 2.0, 'housing': 1.8, 'waste': 1.6},
            'Suburban': {'water': 1.0, 'electricity': 1.2, 'housing': 1.0, 'waste': 1.0},
            'Rural': {'water': 0.7, 'electricity': 0.8, 'housing': 0.6, 'waste': 0.7}
        }

        mult = multipliers[area_type]

        for date in dates:
            # Time-based trends and seasonality
            months_since_start = (date.year - 2019) * 12 + date.month
            growth_factor = 1 + 0.02 * (months_since_start / 12)  # 2% annual growth
            seasonal_factor = 1 + 0.1 * np.sin(2 * np.pi * date.month / 12)  # Seasonal variation

            # Add some noise
            noise = np.random.normal(1, 0.1)

            total_factor = growth_factor * seasonal_factor * noise

            # Calculate demands
            water_demand = base_pop * 150 * mult['water'] * total_factor  # liters/day
            electricity_demand = base_pop * 30 * mult['electricity'] * total_factor  # kWh/day
            housing_units = int(base_pop * 0.3 * mult['housing'] * growth_factor)  # units
            waste_generation = base_pop * 1.2 * mult['waste'] * total_factor  # kg/day

            all_data.append({
                'date': date,
                'grid_id': grid_id,
                'area_type': area_type,
                'water_demand_liters': max(0, water_demand),
                'electricity_demand_kwh': max(0, electricity_demand),
                'housing_units': max(0, housing_units),
                'waste_generation_kg': max(0, waste_generation),
                'population': int(base_pop * growth_factor)
            })

    historical_df = pd.DataFrame(all_data)
    return historical_df

# Generate the data
grid_data = generate_grid_data()
historical_data = generate_historical_demand()

print(f" Generated data for {len(grid_data)} grid cells")
print(f" Historical data: {len(historical_data)} records from 2019-2024")

# Display sample data
print("\n Sample Grid Data:")
print(grid_data.head())
print("\n Sample Historical Demand:")
print(historical_data.head())



def forecast_infrastructure_demand(historical_df, infrastructure_type, grid_id=None, years_ahead=5):
    """Forecast infrastructure demand using Facebook Prophet"""

    # Filter data
    if grid_id:
        data = historical_df[historical_df['grid_id'] == grid_id].copy()
        title_suffix = f" - {grid_id}"
    else:
        # Aggregate all grids
        data = historical_df.groupby('date').agg({
            f'{infrastructure_type}': 'sum'
        }).reset_index()
        title_suffix = " - City Total"

    # Prepare data for Prophet
    prophet_data = pd.DataFrame({
        'ds': data['date'],
        'y': data[f'{infrastructure_type}']
    })

    # Create and fit model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05
    )

    model.fit(prophet_data)

    # Create future dataframe
    future_periods = years_ahead * 12  # monthly forecasts
    future = model.make_future_dataframe(periods=future_periods, freq='M')

    # Make predictions
    forecast = model.predict(future)

    return model, forecast, prophet_data

def plot_forecast(model, forecast, historical_data, infrastructure_type, grid_id=None):
    """Plot forecasting results"""

    # Create subplot
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=[
            f'{infrastructure_type.replace("_", " ").title()} Forecast',
            'Forecast Components'
        ],
        vertical_spacing=0.1
    )

    # Historical data
    fig.add_trace(
        go.Scatter(
            x=historical_data['ds'],
            y=historical_data['y'],
            mode='markers',
            name='Historical Data',
            marker=dict(color='blue', size=4)
        ),
        row=1, col=1
    )

    # Forecast
    fig.add_trace(
        go.Scatter(
            x=forecast['ds'],
            y=forecast['yhat'],
            mode='lines',
            name='Forecast',
            line=dict(color='red', width=2)
        ),
        row=1, col=1
    )

    # Confidence intervals
    fig.add_trace(
        go.Scatter(
            x=forecast['ds'].tolist() + forecast['ds'].tolist()[::-1],
            y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'].tolist()[::-1],
            fill='toself',
            fillcolor='rgba(255,0,0,0.2)',
            line=dict(color='rgba(255,0,0,0)'),
            name='Confidence Interval',
            showlegend=False
        ),
        row=1, col=1
    )

    # Trend component
    fig.add_trace(
        go.Scatter(
            x=forecast['ds'],
            y=forecast['trend'],
            mode='lines',
            name='Trend',
            line=dict(color='green', width=2)
        ),
        row=2, col=1
    )

    # Seasonal component
    if 'yearly' in forecast.columns:
        fig.add_trace(
            go.Scatter(
                x=forecast['ds'],
                y=forecast['yearly'],
                mode='lines',
                name='Seasonal',
                line=dict(color='orange', width=2)
            ),
            row=2, col=1
        )

    fig.update_layout(
        title=f"Infrastructure Demand Forecast: {infrastructure_type.replace('_', ' ').title()}",
        height=800,
        showlegend=True
    )

    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Demand", row=1, col=1)
    fig.update_yaxes(title_text="Component Value", row=2, col=1)

    fig.show()



def create_demand_map(grid_data, forecast_data, infrastructure_type, year=2029):
    """Create interactive map showing predicted demand by grid"""

    print(f"\n🗺️ Creating demand map for {infrastructure_type} in {year}...")

    # Handle case where forecast_data is None
    if forecast_data is not None:
        # Filter forecast data for the target year
        target_date = pd.to_datetime(f'{year}-12-01')
        forecast_subset = forecast_data[forecast_data['ds'] == target_date]

        if forecast_subset.empty:
            print(f" No forecast data for {year}, using latest available")
            forecast_subset = forecast_data.tail(25)  # Get last 25 grids
    else:
        print("ℹ Using simplified demand calculation based on population")

    # Create map centered on the data
    center_lat = grid_data['latitude'].mean()
    center_lon = grid_data['longitude'].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add grid cells with demand information
    for _, grid in grid_data.iterrows():
        # Get forecast for this grid (simplified - using population-based estimate)
        demand_value = grid['population_2024'] * 150  # Simplified calculation

        # Color based on demand level
        if demand_value > 600000:
            color = 'red'
        elif demand_value > 400000:
            color = 'orange'
        else:
            color = 'green'

        folium.CircleMarker(
            location=[grid['latitude'], grid['longitude']],
            radius=min(20, demand_value / 30000),
            popup=f"""
            <b>Grid:</b> {grid['grid_id']}<br>
            <b>Area Type:</b> {grid['area_type']}<br>
            <b>Population:</b> {grid['population_2024']:,}<br>
            <b>Predicted {infrastructure_type}:</b> {demand_value:,.0f}
            """,
            color='black',
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)

    # Add legend
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 150px; height: 90px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; padding: 10px">
    <b>Demand Level</b><br>
    <i class="fa fa-circle" style="color:red"></i> High<br>
    <i class="fa fa-circle" style="color:orange"></i> Medium<br>
    <i class="fa fa-circle" style="color:green"></i> Low
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    return m



def run_comprehensive_analysis():
    """Run complete infrastructure demand analysis"""

    print("\n Running Comprehensive Infrastructure Demand Analysis")
    print("=" * 60)

    # Infrastructure types to analyze
    infrastructure_types = [
        'water_demand_liters',
        'electricity_demand_kwh',
        'housing_units',
        'waste_generation_kg'
    ]

    forecasts = {}

    # Run forecasts for each infrastructure type
    for infra_type in infrastructure_types:
        print(f"\n Analyzing {infra_type.replace('_', ' ').title()}...")

        try:
            model, forecast, historical = forecast_infrastructure_demand(
                historical_data, infra_type, years_ahead=5
            )

            forecasts[infra_type] = {
                'model': model,
                'forecast': forecast,
                'historical': historical
            }

            print(f" Forecast completed for {infra_type}")

            # Plot results
            plot_forecast(model, forecast, historical, infra_type)

        except Exception as e:
            print(f" Error forecasting {infra_type}: {e}")

    # Create summary statistics
    print("\n FORECAST SUMMARY (2029)")
    print("=" * 40)

    for infra_type, data in forecasts.items():
        forecast_2029 = data['forecast'][data['forecast']['ds'] == '2029-12-01']
        if not forecast_2029.empty:
            predicted_value = forecast_2029['yhat'].iloc[0]
            historical_2024 = data['historical']['y'].iloc[-1]
            growth_rate = ((predicted_value - historical_2024) / historical_2024) * 100

            print(f"{infra_type.replace('_', ' ').title()}:")
            print(f"   2024: {historical_2024:,.0f}")
            print(f"   2029: {predicted_value:,.0f}")
            print(f"   Growth: {growth_rate:.1f}%")
            print()

    return forecasts

def create_summary_dashboard(forecasts):
    """Create a summary dashboard of all predictions"""

    # Create subplot for all infrastructure types
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Water Demand (Liters/Day)',
            'Electricity Demand (kWh/Day)',
            'Housing Units',
            'Waste Generation (kg/Day)'
        ]
    )

    positions = [(1,1), (1,2), (2,1), (2,2)]
    colors = ['blue', 'green', 'red', 'orange']

    for i, (infra_type, data) in enumerate(forecasts.items()):
        row, col = positions[i]
        color = colors[i]

        # Add historical data
        fig.add_trace(
            go.Scatter(
                x=data['historical']['ds'],
                y=data['historical']['y'],
                mode='markers',
                name=f'{infra_type} (Historical)',
                marker=dict(color=color, size=3),
                showlegend=False
            ),
            row=row, col=col
        )

        # Add forecast
        fig.add_trace(
            go.Scatter(
                x=data['forecast']['ds'],
                y=data['forecast']['yhat'],
                mode='lines',
                name=f'{infra_type} (Forecast)',
                line=dict(color=color, width=2),
                showlegend=False
            ),
            row=row, col=col
        )

    fig.update_layout(
        title="🏗️ Infrastructure Demand Forecasting Dashboard - SDG 9",
        height=800,
        showlegend=False
    )

    fig.show()


if __name__ == "__main__":
    # Run comprehensive analysis
    forecasts = run_comprehensive_analysis()

    # Create summary dashboard
    if forecasts:
        create_summary_dashboard(forecasts)

        # Create spatial map for one infrastructure type
        try:
            # Use the water demand forecast data
            water_forecast = forecasts.get('water_demand_liters', {}).get('forecast')
            sample_map = create_demand_map(grid_data, water_forecast, 'water_demand', 2029)
            print("\n Interactive map created! (Map will display if running in Jupyter/Colab)")
        except Exception as e:
            print(f"\n Map creation skipped due to error: {e}")
            print("   (This is normal in some environments - forecasting analysis completed successfully)")

        print("\n  ANALYSIS COMPLETE!")
        print("  Key Insights:")
        print("   • Time-series forecasting shows growth trends for all infrastructure")
        print("   • Spatial analysis reveals demand hotspots")
        print("   • 5-year forecasts help with resource planning")
        print("   • Supports SDG 9: Industry, Innovation and Infrastructure")

        # Policy recommendations
        print("\n   POLICY RECOMMENDATIONS:")
        print("     Prioritize infrastructure investment in high-growth areas")
        print("     Plan electricity grid expansion for urban cores")
        print("     Ensure water supply capacity matches projected demand")
        print("     Prepare housing development in suburban areas")
        print("     Scale waste management systems appropriately")

print("\n" + "="*60)
print("PREDICTIVE INFRASTRUCTURE DEMAND SYSTEM READY!")
print("Run the code above to generate forecasts and visualizations")
print("Contributing to SDG 9: Sustainable Infrastructure Development")
print("="*60)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tdk2mce/ha_n7f6j.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tdk2mce/1qlu7mir.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20397', 'data', 'file=/tmp/tmp0tdk2mce/ha_n7f6j.json', 'init=/tmp/tmp0tdk2mce/1qlu7mir.json', 'output', 'file=/tmp/tmp0tdk2mce/prophet_model1pjd_3f4/prophet_model-20250819212349.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
21:23:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


 All libraries imported successfully!
 Predictive Infrastructure Demand Forecasting System
 SDG 9: Industry, Innovation and Infrastructure

 Generating synthetic grid-level data...
 Generating historical demand data (2019-2024)...

 Generating synthetic grid-level data...
 Generated data for 25 grid cells
 Historical data: 1800 records from 2019-2024

 Sample Grid Data:
    grid_id  latitude  longitude  population_2024 area_type
0  GRID_0_0   40.7589   -73.9851             2836  Suburban
1  GRID_0_1   40.7589   -73.9751             3183  Suburban
2  GRID_0_2   40.7589   -73.9651             3529  Suburban
3  GRID_0_3   40.7589   -73.9551             3515  Suburban
4  GRID_0_4   40.7589   -73.9451             2690  Suburban

 Sample Historical Demand:
        date   grid_id area_type  water_demand_liters  electricity_demand_kwh  \
0 2019-01-31  GRID_0_0  Suburban        452377.286947           108570.548867   
1 2019-02-28  GRID_0_0  Suburban        410400.567640            98496.136234

21:23:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


 Forecast completed for water_demand_liters


DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tdk2mce/oc_i6mvu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tdk2mce/nnv2dsia.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83881', 'data', 'file=/tmp/tmp0tdk2mce/oc_i6mvu.json', 'init=/tmp/tmp0tdk2mce/nnv2dsia.json', 'output', 'file=/tmp/tmp0tdk2mce/prophet_modelziekpavw/prophet_model-20250819212349.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
21:23:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


...


 Analyzing Waste Generation Kg...


21:23:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


 Forecast completed for waste_generation_kg



 FORECAST SUMMARY (2029)



🗺️ Creating demand map for water_demand in 2029...
 No forecast data for 2029, using latest available

 Interactive map created! (Map will display if running in Jupyter/Colab)

  ANALYSIS COMPLETE!
  Key Insights:
   • Time-series forecasting shows growth trends for all infrastructure
   • Spatial analysis reveals demand hotspots
   • 5-year forecasts help with resource planning
   • Supports SDG 9: Industry, Innovation and Infrastructure

   POLICY RECOMMENDATIONS:
     Prioritize infrastructure investment in high-growth areas
     Plan electricity grid expansion for urban cores
     Ensure water supply capacity matches projected demand
     Prepare housing development in suburban areas
     Scale waste management systems appropriately

PREDICTIVE INFRASTRUCTURE DEMAND SYSTEM READY!
Run the code above to generate forecasts and visualizations
Contributing to SDG 9: Sustainable Infrastructure Development
